#Package#

In [1]:
import random
import tensorflow as tf
# import pandas as pd
import json
import numpy as np
import pickle
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

lemmatizer = WordNetLemmatizer()

ModuleNotFoundError: No module named 'tensorflow'

#load data#

In [ ]:
tokenizer = Tokenizer()
file_path = '/content/drive/MyDrive/intents.json'

with open(file_path, 'r') as file:
  data_json = json.load(file)

# tag = []
# patterns = []
# responses = []
# for intent in data_json['intents']:
#     tag.extend(intent['tag'])
#     patterns.extend(intent['patterns'])
#     responses.extend(intent['responses'])
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in data_json['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

print(documents)

[(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creators'], 'creator'

#Preprocessing#

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(patterns)
# tokenizer.fit_on_texts(responses)

# #Proses Patterns
# sequences_patterns = tokenizer.texts_to_sequences(patterns)
# max_length_patterns = max(len(seq) for seq in sequences_patterns)
# padded_sequences_patterns = pad_sequences(sequences_patterns, maxlen=max_length_patterns, padding='post', truncating='post')

# #Proses Responses
# sequences_responses = tokenizer.texts_to_sequences(responses)
# max_length_responses = max(len(seq) for seq in sequences_responses)
# padded_sequences_responses = pad_sequences(sequences_responses, maxlen=max_length_responses, padding='post', truncating='post')

# #label Encoding kolom tags
# label_encoder = LabelEncoder()
# tag = label_encoder.fit_transform(tag)

# # Pisahkan data
# X_train, X_test, y_train, y_test = train_test_split(
#     padded_sequences_patterns, padded_sequences_responses, test_size=0.2, random_state=42
# )

# print("Sequences patterns:")
# print(sequences_patterns)

# print("\nPadded Sequences patterns:")
# print(padded_sequences_patterns)

# print("Sequences responses:")
# print(sequences_responses)

# print("\nPadded Sequences responses:")
# print(padded_sequences_responses)

In [ ]:
training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

<ipython-input-5-7785cabc9d9f>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD (learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# model = Sequential()
# model.add(Embedding(total_words, 100, input_length=max_sequence_length))
# model.add(LSTM(100))
# model.add(Dense(total_words_responses, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotModel.h5', hist)

Epoch 1/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1000 - accuracy: 0.9728
Epoch 2/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1344 - accuracy: 0.9605
Epoch 3/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1120 - accuracy: 0.9481
Epoch 4/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1030 - accuracy: 0.9728
Epoch 5/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1100 - accuracy: 0.9654
Epoch 6/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1603 - accuracy: 0.9630
Epoch 7/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1469 - accuracy: 0.9630
Epoch 8/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1793 - accuracy: 0.9605
Epoch 9/200
81/81 [==============================] - 0s 3ms/step - loss: 0.2139 - accuracy: 0.9506
Epoch 10/200
81/81 [==============================] - 0s 3ms/step - loss: 0.1427 - accuracy: 0.9556
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#ChatBot#

In [ ]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('./chatbotModel.h5')
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_word

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []

  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  lsit_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

print('GO!')

GO!
